<a href="https://colab.research.google.com/github/hmyrcmn/FaceMaskDetectionYolo/blob/main/json2yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.7/645.7 kB 14.3 MB/s eta 0:00:00


In [7]:
import os
import json
import numpy as np

def process_all_json_files(input_folder, output_folder):
    # Tüm JSON dosyalarını al
    json_files = [file for file in os.listdir(input_folder) if file.endswith('.json')]

    # Her bir JSON dosyasını işle
    for json_file in json_files:
        json_path = os.path.join(input_folder, json_file)
        convert_json_to_yolov8(json_path, output_folder)

def convert_json_to_yolov8(json_path, output_folder):
    with open(json_path, 'r') as file:
        data = json.load(file)

    os.makedirs(output_folder, exist_ok=True)

    label_mapping = {'boneli': 1, 'bonesiz': 2, 'onluklu': 5, 'maskesiz': 4, 'maskeli': 3, 'onluksuz': 6}  # Etiketleri sayılarla eşleştir

    for shape in data['shapes']:
        label = shape['label']

        if label not in label_mapping:
            continue  # Eğer etiket eşleştirmede yoksa geç

        label_id = label_mapping[label]

        points = shape['points']

        x_values = [point[0] for point in points]
        y_values = [point[1] for point in points]

        x_center = np.mean(x_values)
        y_center = np.mean(y_values)

        min_x = min(x_values)
        max_x = max(x_values)
        min_y = min(y_values)
        max_y = max(y_values)

        width = max_x - min_x
        height = max_y - min_y

        img_width = width  # Gerçek görüntü genişliğiyle değiştirilmesi gerekebilir
        img_height = height  # Gerçek görüntü yüksekliğiyle değiştirilmesi gerekebilir

        x_center_normalized = x_center / img_width
        y_center_normalized = y_center / img_height
        width_normalized = width / img_width
        height_normalized = height / img_height

        # YOLOv8 formatında etiketleme satırını oluştur
        line = '{} {} {} {} {}'.format(label_id, x_center_normalized, y_center_normalized, width_normalized, height_normalized)

        # Dosyaya yaz
        label_filename = json_path.split('/')[-1].split('.json')[0]
        label_path = os.path.join(output_folder, f'{label_filename}.txt')
        with open(label_path, 'a') as f:
            f.write(line + '\n')

if __name__ == "__main__":
    input_folder = '/content/data/'
    output_folder = '/content/out2/'

    process_all_json_files(input_folder, output_folder)


In [8]:
import shutil

def zip_folder(folder_path, output_zip_path):
    shutil.make_archive(output_zip_path, 'zip', folder_path)

if __name__ == "__main__":
    folder_to_zip = '/content/out2/'  # Zip'lenmesini istediğiniz dizin
    output_zip_path = '/content/out2.zip'  # Zip dosyasının çıkış yolu ve adı

    zip_folder(folder_to_zip, output_zip_path)
